In [4]:
import numpy as np

np.random.seed(321)

# Simulation parameters
num_candidates = 1000 # Adjusted number of candidates selected
num_selected = 250  # Adjusted number of candidates selected
num_questions = 200
mean_true_score = 125
std_dev_true_score = 20
num_simulations = 10
N_values = [0, 0.25, 1/3]  # Different negative marking values to simulate
P_values = [10, 20, 30, 40, 50, 60, 70, 80, 90]  # Different percentages of students guessing

# Initialize tables
table1 = np.zeros((len(P_values), len(N_values)))
table2 = np.zeros((len(P_values), len(N_values)))
table3 = np.zeros((len(P_values), len(N_values)))
table4 = np.zeros((len(P_values), len(N_values)))
table5 = np.zeros((len(P_values), len(N_values)))
table6 = np.zeros((len(P_values), len(N_values)))

def simulate_tables123456(P):
    L_values = []
    G_values = []
    T_values = []
    
   
    
    for _ in range(num_simulations):
        d = []
        scores = []
        scores_sorted = []
        selected_scores_list = []
        cut_off_score = []
        L = []
        G = []
        T = []
        # Generate random true scores for all candidates.
        true_scores = np.round(np.random.normal(mean_true_score, std_dev_true_score, num_candidates))
        true_scores = np.clip(true_scores, 0, num_questions).astype(int)
        true_scores_sorted =sorted(true_scores,reverse = True) # a duplicate sorted list to help us find out true ranks
        
        # Generate random scores for all candidates.
        H_values = np.random.binomial(1, P / 100, num_candidates) # create an array of size num_candidates consisting of 0's and 1's.
        A_values = np.random.binomial(num_questions - true_scores, 0.25) # create an array of number of right answers one got by guessing,assuming everyone guesses.
        for i in range(len(N_values)):
            scores.append(true_scores + (H_values * A_values) - (H_values * (num_questions - true_scores - A_values) * N_values[i])) # scores of all candidates in the same order as true_scores.
            scores_sorted.append(sorted(scores[i],reverse = True))
        
        for i in range(num_candidates):
            candidate_data = []
            for j in range(len(N_values)):
                candidate_data.append({'true_score': true_scores[i], 'score': scores[j][i], 'rank': scores_sorted[j].index(scores[j][i]), 'true_rank': true_scores_sorted.index(true_scores[i])})  # Create a dictionary for each cand
            d.append(candidate_data)  # Add the dictionary to the list of candidates
    
        def selected_scores(n,S): # function to choose less than or equal to n people from sorted list of scores in case the nth person and (n+1)th person has the same score. 
            if S[n] != S[n + 1]:
                return S[:n]
            else:
                return selected_scores(n - 1,S)
        selected_scores_true = selected_scores(num_selected,true_scores_sorted)
        cut_off_score_true = selected_scores_true[-1] 
        for i in range(len(N_values)):
            selected_scores_list.append(selected_scores(num_selected,scores_sorted[i]))
            cut_off_score.append(selected_scores_list[i][-1])
        
        # defining L,G,T as defined in the problem.
        # L : number of candidates who should not have been selected, but have been selected
        # G : gap between true cut-off and true score of weakest candidate selected
        # T : true rank of the weakest candidate selected
        for j in range(len(N_values)):
            L.append(len([i for i in range(num_candidates) if d[i][j]['score'] in selected_scores_list[j] and d[i][j]['true_score'] not in selected_scores_true]))
            G.append(cut_off_score_true - min([d[i][j]['true_score'] for i in range(num_candidates) if d[i][j]['score'] in selected_scores_list[j]]))
            T.append(max([d[i][j]['true_rank'] for i in range(num_candidates) if d[i][j]['score'] in selected_scores_list[j]]))
        

        L_values.append(L)
        G_values.append(G)
        T_values.append(T)
    
    avg_L = [0]*len(N_values)
    avg_G = [0]*len(N_values)
    avg_T = [0]*len(N_values)
    fifth_percentile_L = [0]*len(N_values)
    fifth_percentile_G = [0]*len(N_values)
    fifth_percentile_T = [0]*len(N_values)
    
    
    
    # Calculate average values for L,G,T
    for i in range(len(N_values)):
        avg_L[i] = np.mean([L_values[j][i] for j in range(len(L_values))])
        avg_G[i] = np.mean([G_values[j][i] for j in range(len(G_values))])
        avg_T[i] = np.mean([T_values[j][i] for j in range(len(T_values))])
        fifth_percentile_L[i] = np.percentile([L_values[j][i] for j in range(len(L_values))], 5)
        fifth_percentile_G[i] = np.percentile([G_values[j][i] for j in range(len(G_values))], 5)
        fifth_percentile_T[i] = np.percentile([T_values[j][i] for j in range(len(T_values))], 5)

    return [avg_L,avg_G,avg_T,fifth_percentile_L,fifth_percentile_G,fifth_percentile_T]
    
for i in range(len(P_values)):
        P = P_values[i]
        S = simulate_tables123456(P)
        for j in range(len(N_values)):
            table1[i][j] = S[0][j]
            table2[i][j] = S[1][j]
            table3[i][j] = S[2][j]
            table4[i][j] = S[3][j]
            table5[i][j] = S[4][j]
            table6[i][j] = S[5][j]
            
# Print tables with row and column names
print("Table 1. Average L: number of candidates who should not have been selected, but have been selected")
print("P/N\t" + "\t".join(str(n) for n in N_values))
for i, P in enumerate(P_values):
    print(f"{P}\t" + "\t".join(str(val) for val in table1[i]))

print("\nTable 2. Average G: gap between true cut-off and true score of weakest candidate selected")
print("P/N\t" + "\t".join(str(n) for n in N_values))
for i, P in enumerate(P_values):
    print(f"{P}\t" + "\t".join(str(val) for val in table2[i]))

print("\nTable 3. Average T: true rank of the weakest candidate selected")
print("P/N\t" + "\t".join(str(n) for n in N_values))
for i, P in enumerate(P_values):
    print(f"{P}\t" + "\t".join(str(val) for val in table3[i]))

print("\nTable 4. 5th Percentile of L: number of candidates who should not have been selected, but have been selected")
print("P/N\t" + "\t".join(str(n) for n in N_values))
for i, P in enumerate(P_values):
    print(f"{P}\t" + "\t".join(str(val) for val in table4[i]))

print("\nTable 5. 5th Percentile of G: gap between true cut-off and true score of weakest candidate selected")
print("P/N\t" + "\t".join(str(n) for n in N_values))
for i, P in enumerate(P_values):
    print(f"{P}\t" + "\t".join(str(val) for val in table5[i]))

print("\nTable 6. 5th Percentile of T: true rank of the weakest candidate selected")
print("P/N\t" + "\t".join(str(n) for n in N_values))
for i, P in enumerate(P_values):
    print(f"{P}\t" + "\t".join(str(val) for val in table6[i]))
            

            


Table 1. Average L: number of candidates who should not have been selected, but have been selected
P/N	0	0.25	0.3333333333333333
10	30.3	6.4	6.0
20	51.8	11.3	7.6
30	65.3	21.2	12.3
40	64.8	22.5	13.7
50	66.7	26.3	17.1
60	58.8	26.3	19.6
70	46.7	26.1	19.0
80	37.4	25.0	20.2
90	37.5	35.5	30.0

Table 2. Average G: gap between true cut-off and true score of weakest candidate selected
P/N	0	0.25	0.3333333333333333
10	24.2	11.4	5.7
20	22.6	9.8	8.0
30	21.1	12.2	9.2
40	20.3	14.0	9.8
50	15.9	12.7	10.0
60	15.3	11.8	10.3
70	15.7	12.2	10.5
80	12.0	11.4	10.8
90	11.0	11.0	10.2

Table 3. Average T: true rank of the weakest candidate selected
P/N	0	0.25	0.3333333333333333
10	682.3	433.1	328.1
20	646.6	399.4	365.4
30	624.4	445.8	390.0
40	605.4	481.2	396.7
50	518.3	456.4	402.1
60	509.5	440.9	411.0
70	519.1	446.7	413.7
80	448.4	437.0	425.5
90	420.5	420.5	405.8

Table 4. 5th Percentile of L: number of candidates who should not have been selected, but have been selected
P/N	0	0.25	0.3333333333333333
10	24.9	2.